In [ ]:
from nbdev import *
%nbdev_hide_input
!date

Sun Feb 28 09:41:45 PST 2021


# Geminga Analyses

> Ongoing analyses


** `wtlike` library functions used here **
- `get_cells`
- `simulate_cells`
- `get_lightcurve`
- `get_bb_partition`
- `partition_cells`
- `fit_cells`
- `flux_plot`


## Introduction

This notebook does not produce a module to add to the library, it a place to run, and report on analyses using the library. It uses local code, shown below.

This is also an exploration of this style of presenting code, and data analysis based on it.

## Geminga analyses

Following Kerr's example, we use the source Geminga to verify that this analysis of data gives results consistent with its being constant. As you can see below, the BB analysis finds several breaks, which merit further checks. That is followed by a simulation using
Geminga's weight distribution and exposure. The run shown here finds one minor break, a change of 0.08%. 

In [ ]:
%nbdev_collapse_input

from light_curves.tools import *
from wtlike.config import *
from utilities.ipynb_docgen import *
config = Config()
source = PointSource('geminga')

def geminga_combined():
    """
    ### Cache contents for this source
    
    {geminga_cache}
    
    ## Fit to all data
    {outp}
    This is the likelihood, for the combined dataset, a consistency check of the weight calculation.
    {fig1}
    """
        
    with capture_print('contents') as geminga_cache: 
        print(show_cache(config, source.name))
       
    with capture_print('Analysis output') as outp:
        ll = all_data_likelihood(config, source)

    #print(ll,'\nFit: ', ll.fit_info())
    fig1, ax1 = plt.subplots(num=1, figsize=(3,2))
    ll.plot(xlim =( 0.99, 1.01), ax=ax1)
    return locals()
if config.valid:
    nbdoc(geminga_combined)

### Cache contents for this source

<details  class="nbdoc-description" >  <summary> contents </summary>  <div style="margin-left: 5%"><pre>Cache contents for source geminga<br>  key                         size  time                <br>  photons_geminga         31679265  2021-02-28 08:20     <br>  exposure_geminga        86263776  2021-02-28 08:22     <br>  binexp_geminga             64448  2021-02-28 08:22     <br>  cells_geminga            5132223  2021-02-28 08:22     <br>  lightfcurve_geminga      7289195  2021-02-28 08:39     <br>  BB-geminga                   281  2021-02-28 08:43     <br>  weight_hist_geminga          556  2021-02-28 08:43     <br><br></pre></div> </details>

## Fit to all data
<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Cell data for geminga: Restoring from cache with key "cells_geminga"<br></pre></div> </details>
This is the likelihood, for the combined dataset, a consistency check of the weight calculation.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/geminga_combined_fig_01.png" title="images/geminga_combined_fig_01.png">    <img src="images/geminga_combined_fig_01.png" alt="Figure 1 at images/geminga_combined_fig_01.png" >   </a> </figure>
</div>



In [ ]:
%nbdev_collapse_input
def analysis_plots(name, expect=0.9963, short=50, simname=''):
    """
    ### {sim} Geminga data
    
    {which_source} the daily binned data, or cells; then perform a Bayesian Blocks partition;
    finally make fits to the resulting blocks. <br>(Run at {date})
    
    {output}
    
    This shows the fits to all cells, with the BB fit overlays.
    {fig1}
    
    Since this is a constant source, there should be no breaks, that is, only one partition.
    Here is a table of the partition fits:
    
    {df_text}
    The last column represents the compatibility of the flux measurement for each partition
    with the expected value {expect} in equivalent sigma units.
    
    Expand the plot around short, < {short} day partitions.
    {short_check}
    
    {fig2}
    """

    simulated = bool(simname)
    sim= 'Simulated' if simulated else ''
    which_source = 'Simulate a set of ' if simulated else 'Get'
    with capture_print('Analysis output' ) as output:
        if not simulated:
            lc, bb_lc = analyze_data(config, source)
        else: 
            lc, bb_lc = simulation(config, source, bb_key=simname) 

    pd.set_option('display.precision', 3)#, 'display.colheader_justify','left')
    
    df = fit_table(bb_lc, expect=expect)
    df_text = monospace(str(df), 'BB fit table', open=True)
        
    plt.rc('font', size=16)
    fig1, ax = plt.subplots(1,1, sharex=True, figsize=(10,4), num=1)
    bb_overplot(config, lc, bb_lc, ax = ax)
    ax.text(0.05, 0.85, name,  transform=ax.transAxes);
    fig1.width=600

    bb_short = bb_lc.query(f'tw<{short}'); ns =len(bb_short)
    if ns>0:
        short_check=f'There are {ns} such.'
        rows, cols = (ns+2)//3, 3
        fig2, axx = plt.subplots( rows,3,  figsize=(5*cols, 4*rows),
                                 sharey=True, sharex=True,
                     gridspec_kw=dict(top=0.85, left=0.08, bottom=0.15, hspace=0.2 ),num=2)
        if ns>1: fig2.width=600
        axf = axx.flatten()
        [ax.set_visible(False) for ax in axf[ns:]]
        for t, ax in zip(bb_short.t, axf):
            bb_overplot(config, lc, bb_lc, ax=ax, tzero=t, xlim=(-50, +50))
    else:
        fig2=''
        short_check = 'None found.'
    return locals()

In [ ]:
%nbdev_collapse_input
if config.valid:
    nbdoc(analysis_plots, 'Geminga')

In [ ]:
%nbdev_collapse_input
if config.valid:

    nbdoc(analysis_plots, 'Geminga', name='analysis_sim',  simname='analysis_plot_sim', 
         expect=0.9915)

# TO DO
- Fix a 0.4% bias from the cell fit when applied to high-statisics data.

- Look at the intervals detected for the data, of which 5 have measured flux increases around 10%,  more than 4$\sigma$. Possibilities are a problem with the exposure, and a change in the background. The latter can be examined by a 2-D fit with $\beta$ free. Another possibility, fixing $\alpha=0$ and fitting $\beta$ is not (yet) supported.
